# Exercise 1.1

## Install Necessary Packages

In [5]:
%pip install opencv-python numpy

Note: you may need to restart the kernel to use updated packages.



## Import Libraries

In [6]:
import cv2
import numpy as np

## Main Code

In [7]:
# sets a minimum contour area threshold for filtering out contours that are too small to be a car
MIN_CONTOUR_AREA = 4000

In [8]:
video = cv2.VideoCapture("Traffic_Laramie_1.mp4")
assert video.isOpened(), "Can't open the video file"

# initializes a K-nearest neighbors background subtractor with specific parameters such as disabling shadow detection
bg_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows = False, history = 10000, dist2Threshold = 900)

# creates a window with the title "Cam" for displaying the video frames processed
cv2.namedWindow("Cam")

# analyzes the video on a frame-by-frame basis
while True:
    ret, frame = video.read()

    # checks if the video frame was successfully read. If not, it breaks out of the loop
    if not ret:
        break

    # applies Gaussian blur to the frame, removing high-frequency components like noise and edges
    blured_frame = cv2.GaussianBlur(frame, (5, 5), 0)

    # applies the background subtraction algorithm to the blurred frame
    fg_mask = bg_subtractor.apply(blured_frame)

    # applies a threshold to the foreground mask to eliminate shadows
    _, fg_mask = cv2.threshold(fg_mask, 100, 255, cv2.THRESH_BINARY)

    # applies morphological closing operation on the foreground mask 
    # to fill in gaps and smooth the contours of detected objects
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # finds the contours of objects in the foreground mask
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # filters out small contours with an area smaller than the specified minimum contour area
    contours = [c for c in contours if cv2.contourArea(c) > MIN_CONTOUR_AREA]

    # excludes contours in the top half of the frame to focus on cars moving along Main Street
    contours = [c for c in contours if cv2.boundingRect(c)[1] > frame.shape[0] / 2]

    # gets clean contours without holes by applying the convex hull algorithm to each contour in the list
    contours = [cv2.convexHull(c) for c in contours]

    bboxes = []

    # calculates the bounding boxes of the contours and store them in the 'bboxes' list
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        bboxes.append([x, y, x + w, y + h])

    # draws bounding boxes around the detected objects on the frame using the coordinates stored in the 'bboxes' list
    for [x, y, x2, y2] in bboxes:
        cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 1)

    cv2.imshow("Cam", frame)

    # closes the window titled "Cam" when the 'q' key is pressed and exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyWindow("Cam")
        cv2.waitKey(1)
        break